In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import collections
import scipy.stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
from ranking import RankingMeasures

In [17]:
data_cats = ['idea', 'design']
data_keys = {'idea': ['bike', 'cheat', 'meeting', 'night', 'visitor'], 
             'design': ['ai_character', 'olympic']}
data_keys_arr = data_keys['idea'] + data_keys['design']
d_arr = [2, 3]
methods = ['crowdea_%d' % d for d in d_arr]
methods += ['blade_chest_%d_x' % d for d in d_arr] + ['blade_chest_%d_y' % d for d in d_arr]
methods += ['bpr_%d' % d for d in d_arr]
methods += ['crowdbt', 'bt']

In [12]:
# implemented in code/best_ndcg.py
'''
def get_best_ndcg(x, p_truth):
    e = 1
    degrees = np.arange(0, 90.+e, e)
    
    ndcg5_best = -1
    ndcg10_best = -1
    d = x.shape[1]
    assert(d == 2 or d == 3)
    
    if d == 2:
        n = 100
    else:
        n = 1000
        
    if d == 2:
        for r in degrees:
            v_tmp = np.array([np.cos(np.radians(r)), np.sin(np.radians(r))])
            p = x.dot(v_tmp).flatten()
            rm = RankingMeasures(p, p_truth)
            ndcg5 = rm.nDCG(5)
            ndcg10 = rm.nDCG(10)

            if ndcg5 > ndcg5_best:
                ndcg5_best = ndcg5

            if ndcg10 > ndcg10_best:
                ndcg10_best = ndcg10 
            
    else:
        for r in degrees:
            for r_ in degrees:
                v_tmp = np.array([np.sin(np.radians(r))*np.cos(np.radians(r_)),
                                                  np.sin(np.radians(r))*np.sin(np.radians(r_)),
                                                  np.cos(np.radians(r))])
                p = x.dot(v_tmp).flatten()
                rm = RankingMeasures(p, p_truth)
                ndcg5 = rm.nDCG(5)
                ndcg10 = rm.nDCG(10)

                if ndcg5 > ndcg5_best:
                    ndcg5_best = ndcg5

                if ndcg10 > ndcg10_best:
                    ndcg10_best = ndcg10 

    return ndcg5_best, ndcg10_best
    
data_dir = Path('../data/')
result_dir = Path('../result/')

ndcg5_data = []
ndcg10_data = []

for data_cat in data_cats[:1]:
    for data_key in data_keys[data_cat][:1]:
        truth_df = pd.read_csv(data_dir / data_cat / data_key / 'truth.tsv', index_col=0, sep='\t')        
        result_dir_sub = result_dir / data_cat / data_key
        for method in methods:
            print(data_key, method)
            if method == 'bt' or method == 'crowdbt':
                for param in [0.001, 0.01, 0.1]:
                    if method == 'bt':
                        p = np.loadtxt(result_dir_sub / 'bt_x_lambda{}.dat'.format(param))
                    elif method == 'crowdbt':
                        p = np.loadtxt(result_dir_sub / 'crowdbt_x_lambda{}.dat'.format(param))
                        
                    for viewpoint in truth_df.columns:
                        if viewpoint[0] != '*': 
                            continue
                            
                        p_truth = truth_df.loc[:, viewpoint].values
                        rm = RankingMeasures(p, p_truth)
                        ndcg5 = rm.nDCG(5)
                        ndcg10 = rm.nDCG(10)
                        
                        ndcg5_data.append({'data_key': data_key, 'ndcg5': ndcg5, 
                                           'method': method, 'viewpoint': viewpoint, 
                                           'param': param})
                        ndcg10_data.append({'data_key': data_key, 'ndcg10': ndcg10, 
                                            'method': method, 'viewpoint': viewpoint, 
                                            'param': param})                        
                
            elif 'bpr' in method:
                d = int(method.split('_')[-1])
                for param in [0.001, 0.01, 0.1]:
                    if 'bpr_nonneg_unit' in method:
                        x = np.loadtxt(result_dir_sub / 'bpr_x_lambda{}_d{}_nonnegxw_unitw.dat'.format(param, d))
                    elif 'bpr_nonneg' in method:
                        x = np.loadtxt(result_dir_sub / 'bpr_x_lambda{}_d{}_nonnegx.dat'.format(param, d))
                    else:
                        x = np.loadtxt(result_dir_sub / 'bpr_x_lambda{}_d{}.dat'.format(param, d))
                    
                    for viewpoint in truth_df.columns:
                        if viewpoint[0] != '*': 
                            continue

                        p_truth = truth_df.loc[:, viewpoint].values
                        ndcg5, ndcg10 = get_best_ndcg(x, p_truth)

                        ndcg5_data.append({'data_key': data_key, 'ndcg5': ndcg5, 
                                           'method': method, 'viewpoint': viewpoint, 
                                           'param': param})
                        ndcg10_data.append({'data_key': data_key, 'ndcg10': ndcg10, 
                                           'method': method, 'viewpoint': viewpoint, 
                                           'param': param})
                        
            elif 'blade_chest' in method:
                d = int(method.split('_')[-2])
                m = method[-1]
                for param in [0.001, 0.01, 0.1]:
                    if 'blade_chest_dist_nonneg' in method:
                        x = np.loadtxt(result_dir_sub / 'blade_chest_dist_{}_lambda{}_d{}_nonnegxy.dat'.format(m, param, d))
                    elif 'blade_chest_inner_nonneg' in method:
                        x = np.loadtxt(result_dir_sub / 'blade_chest_inner_{}_lambda{}_d{}_nonnegxy.dat'.format(m, param, d))
                    elif 'blade_chest_dist' in method:
                        x = np.loadtxt(result_dir_sub / 'blade_chest_dist_{}_lambda{}_d{}.dat'.format(m, param, d))
                    else:
                        x = np.loadtxt(result_dir_sub / 'blade_chest_inner_{}_lambda{}_d{}.dat'.format(m, param, d))
                    
                    for viewpoint in truth_df.columns:
                        if viewpoint[0] != '*': 
                            continue

                        p_truth = truth_df.loc[:, viewpoint].values
                        ndcg5, ndcg10 = get_best_ndcg(x, p_truth)

                        ndcg5_data.append({'data_key': data_key, 'ndcg5': ndcg5, 
                                           'method': method, 'viewpoint': viewpoint, 
                                           'param': param})
                        ndcg10_data.append({'data_key': data_key, 'ndcg10': ndcg10, 
                                           'method': method, 'viewpoint': viewpoint, 
                                           'param': param})
                
            else:
                d = int(method.split('_')[-1])
                x = np.loadtxt(result_dir_sub / 'crowdea_x_alpha0.1_d{}.dat'.format(d))                    
                
                for viewpoint in truth_df.columns:
                    if viewpoint[0] != '*': 
                        continue
                        
                    p_truth = truth_df.loc[:, viewpoint].values
                    ndcg5, ndcg10 = get_best_ndcg(x, p_truth)
                    
                    ndcg5_data.append({'data_key': data_key, 'ndcg5': ndcg5, 
                                       'method': method, 'viewpoint': viewpoint})
                    ndcg10_data.append({'data_key': data_key, 'ndcg10': ndcg10, 
                                       'method': method, 'viewpoint': viewpoint})
                    
score_dfs = {'ndcg5': pd.DataFrame(ndcg5_data),
            'ndcg10': pd.DataFrame(ndcg10_data)}
'''
print()

In [13]:
score_dfs = {}
score_dfs['ndcg5'] = pd.read_csv('../ndcg/ndcg5_data.csv', index_col=0)
score_dfs['ndcg10'] = pd.read_csv('../ndcg/ndcg10_data.csv', index_col=0)

In [16]:
score

['crowdea_2',
 'crowdea_3',
 'blade_chest_2_x',
 'blade_chest_3_x',
 'blade_chest_2_y',
 'blade_chest_3_y',
 'bpr2',
 'bpr3',
 'crowdbt',
 'bt']

In [18]:
# select best parameters of each baseline for each (measure, data_key)
best_params = collections.defaultdict(dict)

for measure in score_dfs.keys():
    score_df = score_dfs[measure]
    for method in methods:        
        if 'crowdea' in method:
            continue
        avg_df = score_df.loc[score_df.method == method].groupby(['data_key', 'method', 'param']).mean()
        for data_key in data_keys_arr:
            tmp_df = avg_df.loc[(data_key, method)]
            tmp_df = tmp_df.sample(tmp_df.shape[0]) # for tie-breaking
            best_params[measure][(method, data_key)] = tmp_df.sort_values(measure, ascending=False).index[0]
            
for measure in score_dfs.keys():
    score_df = score_dfs[measure]
    for index, row in score_df.iterrows():
        if row.method in methods:
            if 'crowdea' in row.method:
                continue
            if best_params[measure][(row.method, row.data_key)] != row.param:
                score_df.loc[index, measure] = np.nan
    score_df.drop('param', axis=1, inplace=True)
    score_df = score_df.dropna()
    score_dfs[measure] = score_df

In [19]:
avg_dfs = {}
for measure in score_dfs.keys():
    avg_dfs[measure] = score_dfs[measure].groupby(['data_key', 'method']).mean()

In [20]:
# "win" is True if a method performs the best
for measure in score_dfs.keys():
    for d in d_arr:
        avg_df = avg_dfs[measure]
        tmp_methods = ['bt', 'crowdbt'] + [m for m in methods if str(d) in m]
        for data_key in avg_dfs[measure].index.levels[0]:        
            scores = {}
            for method in tmp_methods:
                scores[method] = avg_df.loc[(data_key, method), measure]
            best_score = max(scores.values())
            for method in tmp_methods:
                if '%.3f' % scores[method] == '%.3f' % best_score:
                    avg_df.loc[(data_key, method), 'win_%d' % d] = True
                else:
                    avg_df.loc[(data_key, method), 'win_%d' % d] = False

In [21]:
# "crowdea_stats_win" is True if crowDEA is the statistically significant (p < 0.05) winner by the Wilcoxon signed rank test 
for measure in score_dfs.keys():
    score_df = score_dfs[measure]
    for d in d_arr:
        avg_df = avg_dfs[measure]
        tmp_methods = ['bt', 'crowdbt'] + [m for m in methods if str(d) in m]
        for data_key in avg_dfs[measure].index.levels[0]:
            crowdea_method = [m for m in tmp_methods if 'crowdea' in m][0]
            if avg_df.loc[(data_key, crowdea_method), 'win_%d' % d] == False:
                continue
            crowdea_scores = score_df.loc[(score_df.method == crowdea_method) & (score_df.data_key == data_key), measure].values
            for method in tmp_methods:
                if method == crowdea_method:
                    continue
                else:
                    baseline_scores = score_df.loc[(score_df.method == method) & (score_df.data_key == data_key), measure].values
                    s, p = scipy.stats.wilcoxon(crowdea_scores, baseline_scores)
                    if p <= 0.05:
                        stats_win = True
                    else:
                        stats_win = False
                avg_df.loc[(data_key, method), 'crowdea_stats_win_%d' % d] = stats_win

In [22]:
result_dfs = {}
win_dfs = {}
stats_win_dfs = {}

for measure in avg_dfs.keys():
    for d in d_arr:
        tmp_methods = ['bt', 'crowdbt', 'blade_chest_%d' % d, 'bpr_%d' % d, 'crowdea_%d' % d]

        result_df = pd.DataFrame(index=data_keys_arr, columns=tmp_methods)
        win_df = pd.DataFrame(index=data_keys_arr, columns=tmp_methods)
        stats_win_df = pd.DataFrame(index=data_keys_arr, columns=tmp_methods)

        for data_key in data_keys_arr:
            for method in tmp_methods:
                if 'blade_chest' in method:
                    method_x = method + '_x'
                    method_y = method + '_y'
                    if avg_dfs[measure].loc[(data_key, method_x), measure] > avg_dfs[measure].loc[(data_key, method_y), measure]:
                        method_bc = method_x
                    else:
                        method_bc = method_y
                    result_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method_bc), measure]
                    win_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method_bc), 'win_%d' % d]
                    stats_win_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method_bc), 'crowdea_stats_win_%d' % d]
                else:
                    result_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method), measure]
                    win_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method), 'win_%d' % d]
                    stats_win_df.loc[data_key, method] = avg_dfs[measure].loc[(data_key, method), 'crowdea_stats_win_%d' % d]

        result_dfs[(measure, d)] = result_df
        win_dfs[(measure, d)] = win_df
        stats_win_dfs[(measure, d)] = stats_win_df

---

# nDCG@5 (d=2)

In [23]:
result_dfs[('ndcg5', 2)]

,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,0.77152,0.779258,0.757052,0.827187,0.832811
cheat,0.767964,0.766565,0.813319,0.788554,0.892716
meeting,0.816918,0.814916,0.829405,0.800352,0.876804
night,0.79048,0.789876,0.902675,0.852534,0.917185
visitor,0.818361,0.824591,0.867509,0.932804,0.937662
ai_character,0.902411,0.911642,0.865724,0.928749,0.930365
olympic,0.925774,0.925774,0.919608,0.939931,0.935914


In [24]:
print('Does the method performs the best?')
win_dfs[('ndcg5', 2)]

Does the method performs the best?


,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,False,False,False,False,True
cheat,False,False,False,False,True
meeting,False,False,False,False,True
night,False,False,False,False,True
visitor,False,False,False,False,True
ai_character,False,False,False,False,True
olympic,False,False,False,True,False


In [25]:
print('Is CrowDEA the statistically significant winner?')
stats_win_dfs[('ndcg5', 2)]

Is CrowDEA the statistically significant winner?


,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,True,True,True,False,NaN
cheat,True,True,True,True,NaN
meeting,True,True,True,True,NaN
night,True,True,False,True,NaN
visitor,True,True,True,False,NaN
ai_character,True,True,True,False,NaN
olympic,NaN,NaN,NaN,NaN,NaN


# nDCG@5 (d=3)

In [26]:
result_dfs[('ndcg5', 3)]

,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,0.77152,0.779258,0.802825,0.819293,0.882541
cheat,0.767964,0.766565,0.846592,0.794598,0.923561
meeting,0.816918,0.814916,0.866827,0.888182,0.919762
night,0.79048,0.789876,0.90746,0.913232,0.953478
visitor,0.818361,0.824591,0.916042,0.842319,0.955295
ai_character,0.902411,0.911642,0.904947,0.956587,0.959551
olympic,0.925774,0.925774,0.926532,0.956279,0.965727


In [27]:
print('Does the method performs the best?')
win_dfs[('ndcg5', 3)]

Does the method performs the best?


,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,False,False,False,False,True
cheat,False,False,False,False,True
meeting,False,False,False,False,True
night,False,False,False,False,True
visitor,False,False,False,False,True
ai_character,False,False,False,False,True
olympic,False,False,False,False,True


In [28]:
print('Is CrowDEA the statistically significant winner?')
stats_win_dfs[('ndcg5', 3)]

Is CrowDEA the statistically significant winner?


,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,True,True,True,True,NaN
cheat,True,True,True,True,NaN
meeting,True,True,True,True,NaN
night,True,True,True,True,NaN
visitor,True,True,True,True,NaN
ai_character,True,True,True,False,NaN
olympic,True,True,True,True,NaN


# nDCG@10 (d=2)

In [29]:
result_dfs[('ndcg10', 2)]

,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,0.79844,0.800301,0.755879,0.847357,0.848525
cheat,0.795173,0.790706,0.818961,0.800358,0.895402
meeting,0.823989,0.825414,0.8372,0.817513,0.880205
night,0.808924,0.80832,0.900845,0.861992,0.911928
visitor,0.832233,0.834817,0.873698,0.938092,0.943051
ai_character,0.911494,0.920829,0.864513,0.925894,0.935325
olympic,0.936976,0.936815,0.923008,0.949056,0.947387


In [30]:
print('Does the method performs the best?')
win_dfs[('ndcg10', 2)]

Does the method performs the best?


,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,False,False,False,False,True
cheat,False,False,False,False,True
meeting,False,False,False,False,True
night,False,False,False,False,True
visitor,False,False,False,False,True
ai_character,False,False,False,False,True
olympic,False,False,False,True,False


In [31]:
print('Is CrowDEA the statistically significant winner?')
stats_win_dfs[('ndcg10', 2)]

Is CrowDEA the statistically significant winner?


,bt,crowdbt,blade_chest_2,bpr_2,crowdea_2
bike,True,True,True,False,NaN
cheat,True,True,True,True,NaN
meeting,True,True,True,True,NaN
night,True,True,False,True,NaN
visitor,True,True,True,False,NaN
ai_character,True,True,True,True,NaN
olympic,NaN,NaN,NaN,NaN,NaN


# nDCG@10 (d=3)

In [32]:
result_dfs[('ndcg10', 3)]

,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,0.79844,0.800301,0.796682,0.834982,0.89286
cheat,0.795173,0.790706,0.838802,0.803565,0.926642
meeting,0.823989,0.825414,0.861533,0.890622,0.922563
night,0.808924,0.80832,0.894052,0.910453,0.945
visitor,0.832233,0.834817,0.906219,0.846177,0.951264
ai_character,0.911494,0.920829,0.890526,0.954227,0.953177
olympic,0.936976,0.936815,0.926418,0.952059,0.963585


In [33]:
print('Does the method performs the best?')
win_dfs[('ndcg10', 3)]

Does the method performs the best?


,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,False,False,False,False,True
cheat,False,False,False,False,True
meeting,False,False,False,False,True
night,False,False,False,False,True
visitor,False,False,False,False,True
ai_character,False,False,False,True,False
olympic,False,False,False,False,True


In [34]:
print('Is CrowDEA the statistically significant winner?')
stats_win_dfs[('ndcg10', 3)]

Is CrowDEA the statistically significant winner?


,bt,crowdbt,blade_chest_3,bpr_3,crowdea_3
bike,True,True,True,True,NaN
cheat,True,True,True,True,NaN
meeting,True,True,True,True,NaN
night,True,True,True,True,NaN
visitor,True,True,True,True,NaN
ai_character,NaN,NaN,NaN,NaN,NaN
olympic,True,True,True,True,NaN


---

In [36]:
data_key_tex = {}
for data_key in data_keys_arr:
    data_key_tex[data_key] = data_key.capitalize()
    if data_key == 'ai_character':
        data_key_tex[data_key] = 'Character'

In [37]:
d = 3
tmp_methods = ['bt', 'crowdbt', 'blade_chest_%d' % d, 'bpr_%d' % d, 'crowdea_%d' % d]

lines = []
lines.append('\\begin{tabular}{|l||%s}' % ('c|' * (len(tmp_methods)*2) ))
lines.append('\\hline')

#header1
elems = ['\\multirow{2}{*}{Dataset}']
elems.append('\\multicolumn{%d}{c|}{%s}' % (len(tmp_methods), 'nDCG@5'))
elems.append('\\multicolumn{%d}{c|}{%s}' % (len(tmp_methods), 'nDCG@10'))
lines.append('&'.join(elems) + '\\\\')
lines.append('\\cline{2-6}')
lines.append('\\cline{7-11}')

#header2
elems = ['']
elems.append('%s' % '\\textsc{BT}')
elems.append('%s' % '\\textsc{CrowdBT}')
elems.append('%s' % '\\textsc{Blade-chest}')
elems.append('%s' % '\\textsc{BPR}')
elems.append('%s' % '\\textsc{CrowDEA}')

elems.append('%s' % '\\textsc{BT}')
elems.append('%s' % '\\textsc{CrowdBT}')
elems.append('%s' % '\\textsc{Blade-chest}')
elems.append('%s' % '\\textsc{BPR}')
elems.append('%s' % '\\textsc{CrowDEA}')

lines.append('&'.join(elems) + '\\\\')
lines.append('\\hline')
lines.append('\\hline')

#content
for data_key in data_keys_arr:
    elems = [data_key_tex[data_key]]
    for measure in ['ndcg5', 'ndcg10']:
        result_df = result_dfs[(measure, d)]
        win_df = win_dfs[(measure, d)]
        stats_win_df = stats_win_dfs[(measure, d)]
        for method in tmp_methods:
            v = result_df.loc[data_key, method]
            win = win_df.loc[data_key, method]
            stats_win = stats_win_df.loc[data_key, method]

            if stats_win == True:
                elem = '\\underline{${%.3f}$}' % v
            if win == True:
                elem = '$\\mathbf{%.3f}$' % (v)
            if stats_win != True and win != True:
                elem = '${%.3f}$' % v

            elems.append(elem)
    
    lines.append('&'.join(elems) + '\\\\')

    if data_key == 'visitor':
        lines.append('\\hline')

lines.append('\\hline')
lines.append('\\end{tabular}')

print('\n'.join(lines))

\begin{tabular}{|l||c|c|c|c|c|c|c|c|c|c|}
\hline
\multirow{2}{*}{Dataset}&\multicolumn{5}{c|}{nDCG@5}&\multicolumn{5}{c|}{nDCG@10}\\
\cline{2-6}
\cline{7-11}
&\textsc{BT}&\textsc{CrowdBT}&\textsc{Blade-chest}&\textsc{BPR}&\textsc{CrowDEA}&\textsc{BT}&\textsc{CrowdBT}&\textsc{Blade-chest}&\textsc{BPR}&\textsc{CrowDEA}\\
\hline
\hline
Bike&\underline{${0.772}$}&\underline{${0.779}$}&\underline{${0.803}$}&\underline{${0.819}$}&$\mathbf{0.883}$&\underline{${0.798}$}&\underline{${0.800}$}&\underline{${0.797}$}&\underline{${0.835}$}&$\mathbf{0.893}$\\
Cheat&\underline{${0.768}$}&\underline{${0.767}$}&\underline{${0.847}$}&\underline{${0.795}$}&$\mathbf{0.924}$&\underline{${0.795}$}&\underline{${0.791}$}&\underline{${0.839}$}&\underline{${0.804}$}&$\mathbf{0.927}$\\
Meeting&\underline{${0.817}$}&\underline{${0.815}$}&\underline{${0.867}$}&\underline{${0.888}$}&$\mathbf{0.920}$&\underline{${0.824}$}&\underline{${0.825}$}&\underline{${0.862}$}&\underline{${0.891}$}&$\mathbf{0.923}$\\
Night&\und